In [11]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [2]:
def load_and_prepare_datasets():
    import pandas as pd

    # 🔹 전체 학습 데이터
    df_full = pd.read_csv("피처_43/피처 선택/43_Segment 추가.csv")

    # 🔹 테스트 데이터
    df_test = pd.read_csv("피처_43/피처 선택/test_피처43.csv")

    # 🔹 피처 리스트
    feature_cols = [col for col in df_full.columns if col not in ['ID', 'Segment']]

    # 🔹 dataset1: E vs NotE
    dataset1 = df_full.copy()
    dataset1['Segment'] = dataset1['Segment'].apply(lambda x: 'E' if x == 'E' else 'NotE')

    # 🔹 dataset2: 3클래스용 (C / D / NOT_CD)
    df_no_E = df_full[df_full['Segment'] != 'E'].copy()

    def map_segment_group(x):
        if x == 'C':
            return 'C'
        elif x == 'D':
            return 'D'
        else:
            return 'NOT_CD'  # A, B

    df_no_E['Segment_group'] = df_no_E['Segment'].apply(map_segment_group)
    dataset2 = df_no_E.copy()

    # 🔹 dataset3: A vs B
    df_ab = df_full[df_full['Segment'].isin(['A', 'B'])].copy()
    dataset3 = df_ab.copy()

    # 🔹 그룹별 분리도 유지
    df_c = df_no_E[df_no_E['Segment_group'] == 'C'].copy()
    df_d = df_no_E[df_no_E['Segment_group'] == 'D'].copy()

    # ✅ 리턴값 총 9개
    return df_full, df_test, dataset1, df_c, df_d, df_ab, dataset2, dataset3, feature_cols


In [3]:
def train_model1_kfold(dataset1, feature_cols):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import f1_score
    from catboost import CatBoostClassifier
    import numpy as np

    X = dataset1[feature_cols]
    y = (dataset1['Segment'] == 'E').astype(int)

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []
    best_model = None
    best_score = -1

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostClassifier(
            iterations=10000,
            learning_rate=0.03,
            depth=8,
            eval_metric='F1',
            random_seed=42,
            task_type='GPU',
            verbose=200,
            early_stopping_rounds=1000
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val))

        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='micro')
        f1_scores.append(f1)
        print(f"✅ model1 | Fold {fold} F1_micro: {f1:.4f}")

        if f1 > best_score:
            best_score = f1
            best_model = model

    print(f"\n✅ model1 평균 F1_micro (5-fold): {np.mean(f1_scores):.4f}")
    best_model.save_model("방법 3/model/model1_E_vs_NotE.cbm")
    return best_model

In [4]:
df_full, df_test, dataset1, df_c, df_d, df_ab, dataset2, dataset3, feature_cols = load_and_prepare_datasets()


In [5]:
model1 = train_model1_kfold(dataset1, feature_cols)


0:	learn: 0.9375989	test: 0.9374887	best: 0.9374887 (0)	total: 49.3ms	remaining: 8m 13s
200:	learn: 0.9520283	test: 0.9516776	best: 0.9516776 (200)	total: 4.59s	remaining: 3m 43s
400:	learn: 0.9554065	test: 0.9549602	best: 0.9549602 (400)	total: 9.12s	remaining: 3m 38s
600:	learn: 0.9573727	test: 0.9567329	best: 0.9567329 (600)	total: 13.5s	remaining: 3m 31s
800:	learn: 0.9587671	test: 0.9579346	best: 0.9579391 (798)	total: 18s	remaining: 3m 26s
1000:	learn: 0.9598080	test: 0.9587513	best: 0.9587513 (1000)	total: 22.6s	remaining: 3m 22s
1200:	learn: 0.9606989	test: 0.9593856	best: 0.9593856 (1200)	total: 27s	remaining: 3m 17s
1400:	learn: 0.9615568	test: 0.9598608	best: 0.9598608 (1400)	total: 31.5s	remaining: 3m 13s
1600:	learn: 0.9622279	test: 0.9603534	best: 0.9603534 (1600)	total: 36s	remaining: 3m 8s
1800:	learn: 0.9628572	test: 0.9607759	best: 0.9607759 (1800)	total: 40.5s	remaining: 3m 4s
2000:	learn: 0.9634592	test: 0.9611408	best: 0.9611408 (2000)	total: 45s	remaining: 2m 59s


KeyboardInterrupt: 

In [ ]:
def train_model2_kfold(dataset2, feature_cols):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import f1_score
    from catboost import CatBoostClassifier
    import numpy as np

    X = dataset2[feature_cols]
    y = dataset2['Segment_group']  # 다중 클래스

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []
    best_model = None
    best_score = -1

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostClassifier(
            iterations=10000,
            learning_rate=0.03,
            depth=8,
            loss_function='MultiClass',
            eval_metric='TotalF1',
            random_seed=42,
            task_type='GPU',
            verbose=200,
            early_stopping_rounds=1000
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val))

        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='micro')
        f1_scores.append(f1)
        print(f"✅ model2 | Fold {fold} F1_micro: {f1:.4f}")

        if f1 > best_score:
            best_score = f1
            best_model = model

    print(f"\n✅ model2 평균 F1_micro (3-fold): {np.mean(f1_scores):.4f}")
    best_model.save_model("C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/model/model2_3class.cbm")
    return best_model


In [ ]:
model2 = train_model2_kfold(dataset2, feature_cols)

In [ ]:
def train_model3_kfold(dataset3, feature_cols):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import f1_score
    from catboost import CatBoostClassifier
    import numpy as np

    X = dataset3[feature_cols]
    y = dataset3['Segment']  # 값: A or B

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []
    best_model = None
    best_score = -1

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = CatBoostClassifier(
            iterations=10000,
            learning_rate=0.03,
            depth=8,
            loss_function='Logloss',
            eval_metric='F1',
            random_seed=42,
            task_type='GPU',
            verbose=200,
            early_stopping_rounds=1000
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val))

        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='micro')
        f1_scores.append(f1)
        print(f"✅ model3 | Fold {fold} F1_micro: {f1:.4f}")

        if f1 > best_score:
            best_score = f1
            best_model = model

    print(f"\n✅ model3 평균 F1_micro (3-fold): {np.mean(f1_scores):.4f}")
    best_model.save_model("C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/model/model3_A_vs_B.cbm")
    return best_model


In [ ]:
model3 = train_model3_kfold(dataset3, feature_cols)

In [ ]:
from catboost import CatBoostClassifier

# 🔹 model1: E vs NotE
model1 = CatBoostClassifier()
model1.load_model("C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/model/model1_E_vs_NotE.cbm")

# 🔹 model2: C / D / NOT_CD (3-class)
model2 = CatBoostClassifier()
model2.load_model("C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/model/model2_3class.cbm")

# 🔹 model3: A vs B
model3 = CatBoostClassifier()
model3.load_model("C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/model/model3_A_vs_B.cbm")


In [ ]:
# 🔹 model1: E vs NotE
pred1 = model1.predict(X_test)
df_stage1 = df_test.copy()
df_stage1['Segment'] = pred1
df_e = df_stage1[df_stage1['Segment'] == 'E']
df_not_e = df_stage1[df_stage1['Segment'] != 'E']
X_not_e = X_test.loc[df_not_e.index]

# 🔹 model2: C, D, NOT_CD
pred2 = model2.predict(X_not_e)
df_stage2 = df_not_e.copy()
df_stage2['Segment'] = pred2
df_cd = df_stage2[df_stage2['Segment'].isin(['C', 'D'])]
df_not_cd = df_stage2[df_stage2['Segment'] == 'NOT_CD']
X_not_cd = X_not_e.loc[df_not_cd.index]

# 🔹 model3: A vs B
pred3 = model3.predict(X_not_cd)
df_stage3 = df_not_cd.copy()
df_stage3['Segment'] = pred3
df_ab = df_stage3


In [ ]:
final_result = pd.concat([df_e, df_cd, df_ab], axis=0)
final_result = final_result[['ID', 'Segment']].reset_index(drop=True)

# 저장
final_result.to_csv(
    "C:/Users/PC/Desktop/부트캠프/workspace/FinalProject/주피터 파일/모델분할/피처_43_전체/피처_43/피처 선택/final_segment_prediction.csv",
    index=False
)


In [7]:
final_result['ID'].duplicated().sum()  # ✅ 반드시 0 나와야 함

NameError: name 'final_result' is not defined

In [6]:
print(final_result['Segment'].value_counts())

NameError: name 'final_result' is not defined

---

In [12]:
from catboost import CatBoostClassifier
import pandas as pd

# 모델 경로
base_path = "방법 3/model"

model1 = CatBoostClassifier()
model1.load_model(f"{base_path}/model1_E_vs_NotE.cbm")

model2 = CatBoostClassifier()
model2.load_model(f"{base_path}/model2_3class.cbm")

model3 = CatBoostClassifier()
model3.load_model(f"{base_path}/model3_A_vs_B.cbm")


In [13]:
df_test = pd.read_csv("피처_43/피처 선택/test_피처43.csv")

# 불필요한 컬럼 제거 (ID_1, Unnamed 등)
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]
df_test = df_test.loc[:, ~df_test.columns.str.startswith('ID_') | (df_test.columns == 'ID')]

# feature_cols 확인 (ID, Segment 제외)
feature_cols = [col for col in df_test.columns if col not in ['ID', 'Segment']]
X_test = df_test[feature_cols]


In [22]:
# 1단계: E vs NotE
pred1 = model1.predict(X_test)
df_stage1 = df_test.copy()
df_stage1['Segment'] = pred1
df_e = df_stage1[df_stage1['Segment'] == 'E']
df_not_e = df_stage1[df_stage1['Segment'] != 'E']
X_not_e = X_test.loc[df_not_e.index]

# 2단계: C / D / NOT_CD
pred2 = model2.predict(X_not_e)
df_stage2 = df_not_e.copy()
df_stage2['Segment'] = pd.Series(pred2.ravel(), index=X_not_e.index)
df_cd = df_stage2[df_stage2['Segment'].isin(['C', 'D'])]
df_not_cd = df_stage2[df_stage2['Segment'] == 'NOT_CD']
X_not_cd = X_not_e.loc[df_not_cd.index]

# 3단계: A vs B (NOT_CD 대상만)
pred3 = model3.predict(X_not_cd)
df_stage3 = df_not_cd.copy()
df_stage3['Segment'] = pred3  # 여기서 반드시 NOT_CD → A/B로 덮어씌움

# 최종 결합
final_result = pd.concat([df_e, df_cd, df_stage3], axis=0)
final_result = final_result[['ID', 'Segment']].reset_index(drop=True)